⚔️ Side Quest Notebook: Imputation Optimization ⚔️
==============================================================

**Author:** Xavier R Nogueira

**Overview:** In my first competition notebook, `NB1_PreProcessing_Data.ipynb`, missing values in the Protein and Peptide training datasets were imputed using both Iterative and KNN imputation. That notebook will remain the first notebook in my workflow, however, in this notebook we will explore whether our imputation accuracy can be improved for each method via altering parameters. In later notebooks we will make predictions using training data filled with both methods, and evaluate results at the prediction task level.

**Methodology:**
1. Pull in the columnar formatted `protein_data_raw.parquet` and `peptide_data_raw.parquet` training data files into `pd.DataFrame`s. Combine them into one table.
2. Combine the Protein/Peptide boolean missing data masks. Make a dictionary that returns indices where there IS data for a given column.
3. Set up a version of K-Fold CV where a different subset of cells are coverted to `np.nan` in each fold such that all non-empty cells get converted just once. Evaluate imputation accuracy.
4. Run `Optuna` evaluation for both imputation methods across their parameter space.
5. Record all results in a `pd.DataFrame` such that if we eliminate features later, we can focus on the imputation method that provides the best performance for our subselection of columns.

In [52]:
# core imports
import pandas as pd
import numpy as np
import hvplot.pandas
from typing import (
    List,
    Dict,
    Optional,
)

# enable experimental imputer
from sklearn.experimental import enable_iterative_imputer

# import our imputation algos
from sklearn.impute import (
    IterativeImputer,
    KNNImputer,
)

# Pull in data

## Combine raw data tables

In [3]:
# load in data from parquet
proteins_df = pd.read_parquet(
    'prepped_inputs/protein_data_raw.parquet',
    engine='pyarrow',
)
peptide_df = pd.read_parquet(
    'prepped_inputs/peptide_data_raw.parquet',
    engine='pyarrow',
)

In [4]:
# keep track of our protein / peptide columns
protein_cols = proteins_df.columns
peptide_cols = peptide_df.columns

# join the protein / peptide data
prot_and_peps_df = pd.concat(
    [proteins_df, peptide_df],
    axis=1,
)

In [5]:
prot_and_peps_df.head()

,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,O60888,O75144,O75326,O94919,P00441,P00450,P00734,P00736,P00738,P00746,P00747,P00748,P00751,P01008,P01009,P01011,P01019,P01023,P01024,P01031,P01033,P01034,P01042,P01344,P01591,P01594,P01608,P01621,P01717,P01780,P01833,P01834,P01857,P01859,P01860,P01861,P01876,P01877,P02452,P02647,P02649,P02652,P02655,P02656,P02671,P02675,P02679,P02747,P02748,P02749,P02750,...,VSPTDC(UniMod_4)SAVEPEAEK,VSTLPAITLK,VTAAPQSVC(UniMod_4)ALR,VTEIWQEVMQR,VTEPISAESGEQVER,VTGVVLFR,VTIKPAPETEKRPQDAK,VTIPTDLIASSGDIIK,VTLTC(UniMod_4)VAPLSGVDFQLR,VTSIQDWVQK,VTTVASHTSDSDVPSGVTEVVVK,VVEESELAR,VVEQMC(UniMod_4)ITQYER,VVVNFAPTIQEIK,VYAC(UniMod_4)EVTHQGLSSPVTK,VYC(UniMod_4)DMNTENGGWTVIQNR,VYTVDLGR,WC(UniMod_4)AVSEHEATK,WEAEPVYVQR,WELALGR,WGYC(UniMod_4)LEPK,WKNFPSPVDAAFR,WLPSSSPVTGYR,WQEEMELYR,WSGQTAIC(UniMod_4)DNGAGYC(UniMod_4)SNPGIPIGTR,WSRPQAPITGYR,WSSTSPHRPR,WYEIEKIPTTFENGR,WYFDVTEGK,YAMVYGYNAAYNR,YANC(UniMod_4)HLAR,YFIDFVAR,YGFIEGHVVIPR,YGLDSDLSC(UniMod_4)K,YGLVTYATYPK,YGQTIRPIC(UniMod_4)LPC(UniMod_4)TEGTTR,YHDRDVWKPEPC(UniMod_4)R,YIETDPANR,YIFHNFMER,YIVSGTPTFVPYLIK,YKAAFTEC(UniMod_4)C(UniMod_4)QAADK,YLFLNGNK,YLGEEYVK,YLQEIYNSNNQK,YLYEIAR,YNSQNQSNNQFVLYR,YPGPQAEGDSEGLSQGLVDREK,YPNC(UniMod_4)AYR,YPSLSIHGIEGAFDEPGTK,YQC(UniMod_4)YC(UniMod_4)YGR,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
visit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10053_0,9104.27,402321.0,NaN,NaN,7150.57,2497.84,83002.9,15113.6,167327.0,129048.0,53069.5,NaN,11074.6,NaN,774736.0,474672.0,NaN,3594820.0,34217.2,365510.0,28713.8,475601.0,1849090.0,12825300.0,1084770.0,1197290.0,1005230.0,2669740.0,NaN,79917.7,18811700.0,541909.0,120502.0,8277.92,NaN,8776.14,196510.0,88369.3,12877.0,NaN,2440800.0,3750260.0,2238830.0,49324.7,83537.6,1874270.0,75415.7,18309.7,4032650.0,5158030.0,123201.0,NaN,33735.9,413906.0,336093.0,189063.0,117286.0,57607.3,1000890.0,194973.0,...,49307.3,86217.6,437354.0,85433.3,2497.84,NaN,NaN,85231.1,66733.6,339999.0,1579800.0,NaN,292483.0,43339.3,1296660.0,61108.9,NaN,561125.0,19881.1,NaN,NaN,594013.0,176748.0,187165.0,19940.3,117771.0,NaN,723167.0,30318.9,NaN,NaN,NaN,102861.0,229992.0,153939.0,127619.0,NaN,NaN,NaN,11505.5,2831940.0,19533.7,1747090.0,34810.6,24442300.0,29038.9,114029.0,NaN,103946.0,NaN,202274.0,NaN,4401830.0,77482.6,583075.0,76705.7,104260.0,530223.0,NaN,7207.30
10053_12,10464.20,435586.0,NaN,NaN,NaN,NaN,197117.0,15099.1,164268.0,108114.0,55856.4,NaN,44516.3,NaN,1025080.0,391328.0,NaN,1992590.0,119396.0,365969.0,23347.6,406037.0,1769370.0,11871200.0,1135570.0,1178170.0,1230990.0,3360790.0,NaN,78499.7,19343900.0,584371.0,76739.9,7124.72,14098.50,7592.37,185831.0,85195.6,5802.1,1860110.0,2030500.0,4872420.0,2165970.0,131029.0,178106.0,1800070.0,11026.3,30729.5,5111760.0,5201730.0,NaN,17404.8,43910.3,562354.0,349269.0,156655.0,106468.0,60908.1,871500.0,236112.0,...,19065.9,78750.3,437547.0,126827.0,NaN,NaN,NaN,62315.8,97854.4,195976.0,1616750.0,NaN,289648.0,51576.8,996287.0,55446.7,1860110.0,NaN,23515.0,NaN,NaN,681018.0,189668.0,237930.0,18230.6,123852.0,NaN,617841.0,60135.0,NaN,NaN,NaN,114651.0,211126.0,182877.0,112425.0,NaN,NaN,NaN,30119.8,2527260.0,26771.7,1837800.0,28912.5,23326600.0,37132.6,143767.0,NaN,118192.0,NaN,201009.0,NaN,5001750.0,36745.3,355643.0,92078.1,123254.0,453883.0,49281.9,25332.80
10053_18,13235.70,507386.0,7126.96,24525.7,NaN,2372.71,126506.0,16289.6,168107.0,163776.0,74672.3,NaN,57117.2,53157.9,1104540.0,491425.0,75603.2,2683290.0,57081.2,309751.0,89467.3,523401.0,2821870.0,14319900.0,1444430.0,1597540.0,1333940.0,3728860.0,7907.66,87682.5,18772400.0,971184.0,94953.3,10847.80,13656.10,14548.20,89982.2,87116.5,18800.8,3372600.0,2106740.0,4331390.0,1420530.0,1191760.0,150159.0,1508300.0,92023.6,60339.2,5713180.0,8381290.0,245486.0,20860.6,27293.8,545170.0,565451.0,181571.0,1

## Combine missing value matrices

In [6]:
# load in data from parquet
proteins_mask_df = pd.read_parquet(
    'prepped_inputs/protein_data_missing_values_mask.parquet',
    engine='pyarrow',
)
peptide_mask_df = pd.read_parquet(
    'prepped_inputs/peptide_data_missing_values_mask.parquet',
    engine='pyarrow',
)

In [8]:
# join the protein / peptide data
bool_mask_df = pd.concat(
    [proteins_mask_df, peptide_mask_df],
    axis=1,
)
bool_mask_df.head()

,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,O60888,O75144,O75326,O94919,P00441,P00450,P00734,P00736,P00738,P00746,P00747,P00748,P00751,P01008,P01009,P01011,P01019,P01023,P01024,P01031,P01033,P01034,P01042,P01344,P01591,P01594,P01608,P01621,P01717,P01780,P01833,P01834,P01857,P01859,P01860,P01861,P01876,P01877,P02452,P02647,P02649,P02652,P02655,P02656,P02671,P02675,P02679,P02747,P02748,P02749,P02750,...,VSPTDC(UniMod_4)SAVEPEAEK,VSTLPAITLK,VTAAPQSVC(UniMod_4)ALR,VTEIWQEVMQR,VTEPISAESGEQVER,VTGVVLFR,VTIKPAPETEKRPQDAK,VTIPTDLIASSGDIIK,VTLTC(UniMod_4)VAPLSGVDFQLR,VTSIQDWVQK,VTTVASHTSDSDVPSGVTEVVVK,VVEESELAR,VVEQMC(UniMod_4)ITQYER,VVVNFAPTIQEIK,VYAC(UniMod_4)EVTHQGLSSPVTK,VYC(UniMod_4)DMNTENGGWTVIQNR,VYTVDLGR,WC(UniMod_4)AVSEHEATK,WEAEPVYVQR,WELALGR,WGYC(UniMod_4)LEPK,WKNFPSPVDAAFR,WLPSSSPVTGYR,WQEEMELYR,WSGQTAIC(UniMod_4)DNGAGYC(UniMod_4)SNPGIPIGTR,WSRPQAPITGYR,WSSTSPHRPR,WYEIEKIPTTFENGR,WYFDVTEGK,YAMVYGYNAAYNR,YANC(UniMod_4)HLAR,YFIDFVAR,YGFIEGHVVIPR,YGLDSDLSC(UniMod_4)K,YGLVTYATYPK,YGQTIRPIC(UniMod_4)LPC(UniMod_4)TEGTTR,YHDRDVWKPEPC(UniMod_4)R,YIETDPANR,YIFHNFMER,YIVSGTPTFVPYLIK,YKAAFTEC(UniMod_4)C(UniMod_4)QAADK,YLFLNGNK,YLGEEYVK,YLQEIYNSNNQK,YLYEIAR,YNSQNQSNNQFVLYR,YPGPQAEGDSEGLSQGLVDREK,YPNC(UniMod_4)AYR,YPSLSIHGIEGAFDEPGTK,YQC(UniMod_4)YC(UniMod_4)YGR,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
visit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10053_0,False,False,True,True,False,False,False,False,False,False,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,True,False,False,True,True,False,False,False,False,False,True,False,False,True,True,True,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,True,False
10053_12,False,False,True,True,True,True,False,False,False,False,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,True,False,False,False,False,True,False,False,False,False,False,True,False,True,True,False,False,False,False,False,True,False,False,True,True,True,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False
10053_18,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
10138_12,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,F

## Make a dictionary containing column headers as keys, and non-empty cell indices as values

In [17]:
%%time
values_exist_dict = {}
for col in bool_mask_df.columns:
    values_exist_dict[col] = list(
        bool_mask_df.loc[
            (bool_mask_df[col] == False)
        ].index
    )
len(values_exist_dict)

Wall time: 1.7 s


1195

# Define functions for evaluation

**Note:** The following is a modified version of our main workflow defined in `Tabular_MachineLearning_Projects/ml_models`.

## Define function to run full matrix K-Fold

This function will need to randomly convert some proportion (1/K) of real cell values to NaN for each trial, without repeating the same cell twice.

In [24]:
df = prot_and_peps_df.copy()
num_values_to_convert = df.apply(lambda x: len(x.dropna()) // 5)
num_values_to_convert.sort_values()

QALPQVR                    97
Q99829                     97
EPQVYTLPPSRDELTK          112
TPSGLYLGTC(UniMod_4)ER    118
SLEDQVEMLR                119
                         ... 
P00751                    222
P02749                    222
P02750                    222
P12109                    222
P36222                    222
Length: 1195, dtype: int64

In [28]:
np.random.choice(values_exist_dict['P00751'])

'42003_0'

In [47]:
choose_from_dict = values_exist_dict.copy()

In [ ]:
# Set the values to NaN in the DataFrame at the selected indices
df.values[~converted_rows, np.arange(len(df.columns))] = np.nan

In [ ]:
def get_fold_matrix(
    data_df: pd.pd.DataFrame,
    values_exist_dict: Dict[str, List[str]],
) -> pd.DataFrame:
    pass

In [56]:
def k_fold_cv(
    data_df: pd.DataFrame,
    values_exist_dict: Dict[str, List[str]],
    results_df: Optional[pd.DataFrame] = None,
):
    # find the number of values to include in our folds for each column
    num_values_to_convert = data_df.apply(lambda x: len(x.dropna()) // 5)

    # make a copy of our values_exist_dict to choose from
    choose_from_dict = values_exist_dict.copy()

    # set up true vs predicted values dict
    true_preds_dict = {}

    for fold in range(kfolds):
        # get the corresponding valid value indices for each column, excluding converted rows
        indices_to_convert = np.array(
            [np.random.choice(
                choose_from_dict[col],
                size=num_values_to_convert[col] - 1,
                replace=False,
            ) for col in list(choose_from_dict.keys())
            ],
            dtype='object',
        ).T

        # 


In [54]:
# find the number of values to include in our folds for each column
num_values_to_convert = df.apply(lambda x: len(x.dropna()) // 5)

# get the corresponding valid value indices for each column, excluding converted rows
indices_to_convert = np.array(
    [np.random.choice(
        values_exist_dict[col],
        size=num_values_to_convert[col],
        replace=False,
        #p=prob_dist[col],
    ) for col in list(values_exist_dict.keys())
    ],
    dtype='object',
).T

zipped_dict = dict(zip(values_exist_dict.keys(), indices_to_convert))